## SDG indicator 11.3.1: Ratio of land consumption rate to population growth rate

- #### import datasets: 
     - #### mid-year opulation estimates from ONS for latest year (e.g 2019) and previous year (3 years before)
     - #### land area datasets from OS for latest year (e.g 2019) and previous year (3 years before)
- #### check LAD boundary changes between the 2year periods
- #### calculate population growth rate
- #### calculate land consumption rate
- #### calculate Ratio of land consumption rate to population growth rate

In [79]:
# import libraries
from pathlib import Path
import geopandas as gpd
import pandas as pd
from pandas import DataFrame, merge
import numpy as np
import matplotlib.pyplot as plt
import shapely
import csv
import math
import os
print(Path('.').resolve())

C:\Users\shavar


In [80]:
#calculate Population Growth rate (PGR) = LN(Pop_t+n/Pop_t )/(y)
     # Where: LN is the natural logarithm value
         # Pop_t is the total population within the urban area/city in the past/initial year
         # Pop_t+n is the total population within the urban area/city in the current/final year
         # y is the number of years between the two measurement periods

# read population data 
    # make sure all columns with numbers have float dtype by using (df = pd.read_csv(<PATH>, thousands=',')
pop_est16ew = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\population\SAPE20DT1_mid2016_lsoa_ew.csv',thousands=',').set_index('LSOA11CD')
pop_est19ew = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\population\SAPE22DT2_mid2019_lsoa_ew.csv',thousands=',').set_index('LSOA11CD')
pop_est16sc = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\population\SAPE16_mid2016_dz_sc.csv',thousands=',').set_index('DZ2011CD')
pop_est19sc = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\population\SAPE19_mid2019_dz_sc.csv',thousands=',').set_index('DZ2011CD')

    # remove whitespace before and after column names
pop_est16ew.columns = pop_est16ew.columns.str.strip()
pop_est19ew.columns = pop_est19ew.columns.str.strip()
pop_est16sc.columns = pop_est16sc.columns.str.strip()
pop_est19sc.columns = pop_est19sc.columns.str.strip()

display(pop_est16ew.head())
display(pop_est19ew.head())
display(pop_est16sc.head())
display(pop_est19sc.head())


    # merge population tables for england and wales and for scotland for 2016 and 2019
pop_est19_16ew = pd.merge(pop_est19ew, pop_est16ew, on='LSOA11CD', how='outer', indicator=True, suffixes=('_19','_16'))
pop_est19_16sc = pd.merge(pop_est19sc, pop_est16sc, on='DZ2011CD', how='outer', indicator=True, suffixes=('_19','_16'))

    # change data type from string to float for columns with population data
#pop_est19_16ew['All_Ages_2016']=pop_est19_16ew['All_Ages_2016'].str.replace(',','').astype(np.float32)
#pop_est19_16ew['All_Ages_2019']=pop_est19_16ew['All_Ages_2019'].str.replace(',','').astype(np.float32)
#pop_est19_16sc['All_Ages_2016']=pop_est19_16sc['All_Ages_2016'].str.replace(',','').astype(np.float32)
#pop_est19_16sc['All_Ages_2019']=pop_est19_16sc['All_Ages_2019'].str.replace(',','').astype(np.float32)

    #check if merge operation has run successfully
display(pop_est19_16ew.head())
display(pop_est19_16sc.head())

#calculate Population Growth rate (PGR) = LN(Pop_t+n/Pop_t )/(y)

    #create new column (pgr19_16) and calculate PGR
pop_est19_16ew['pgr19_16'] = np.log(pop_est19_16ew['All_Ages_2019']/pop_est19_16ew['All_Ages_2016'])/3
pop_est19_16sc['pgr19_16'] = np.log(pop_est19_16sc['All_Ages_2019']/pop_est19_16sc['All_Ages_2016'])/3

    #check if code has run successfully
display(pop_est19_16ew['pgr19_16'].head())
display(pop_est19_16sc['pgr19_16'].head())

    # write table with pgr calculation to file
pop_est19_16ew.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\pgr_19_16ew.csv')
pop_est19_16sc.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\pgr_19_16sc.csv')

    




,LSOA11NM,LAD16CD,LAD16NM,All_Ages_2016
LSOA11CD,,,,
E01031349,Adur 001A,E07000223,Adur,1311
E01031350,Adur 001B,E07000223,Adur,1182
E01031351,Adur 001C,E07000223,Adur,1576
E01031352,Adur 001D,E07000223,Adur,1544
E01031370,Adur 001E,E07000223,Adur,1494


,LSOA11NM,LAD19CD,LAD19NM,All_Ages_2019
LSOA11CD,,,,
E01011949,Hartlepool 009A,E06000001,Hartlepool,1954
E01011950,Hartlepool 008A,E06000001,Hartlepool,1257
E01011951,Hartlepool 007A,E06000001,Hartlepool,1209
E01011952,Hartlepool 002A,E06000001,Hartlepool,1740
E01011953,Hartlepool 002B,E06000001,Hartlepool,2033


,DZ11NM,CA16CD,CA16NM,All_Ages_2016
DZ2011CD,,,,
S01006506,Culter - 01,S12000033,Aberdeen City,897
S01006507,Culter - 02,S12000033,Aberdeen City,807
S01006508,Culter - 03,S12000033,Aberdeen City,640
S01006509,Culter - 04,S12000033,Aberdeen City,555
S01006510,Culter - 05,S12000033,Aberdeen City,671


,DZ11NM,CA19CD,CA19NM,All_Ages_2019
DZ2011CD,,,,
S01006506,Culter - 01,S12000033,Aberdeen City,867
S01006507,Culter - 02,S12000033,Aberdeen City,780
S01006508,Culter - 03,S12000033,Aberdeen City,615
S01006509,Culter - 04,S12000033,Aberdeen City,508
S01006510,Culter - 05,S12000033,Aberdeen City,649


,LSOA11NM_19,LAD19CD,LAD19NM,All_Ages_2019,LSOA11NM_16,LAD16CD,LAD16NM,All_Ages_2016,_merge
LSOA11CD,,,,,,,,,
E01011949,Hartlepool 009A,E06000001,Hartlepool,1954,Hartlepool 009A,E06000001,Hartlepool,1966,both
E01011950,Hartlepool 008A,E06000001,Hartlepool,1257,Hartlepool 008A,E06000001,Hartlepool,1389,both
E01011951,Hartlepool 007A,E06000001,Hartlepool,1209,Hartlepool 007A,E06000001,Hartlepool,1217,both
E01011952,Hartlepool 002A,E06000001,Hartlepool,1740,Hartlepool 002A,E06000001,Hartlepool,1755,both
E01011953,Hartlepool 002B,E06000001,Hartlepool,2033,Hartlepool 002B,E06000001,Hartlepool,2040,both


,DZ11NM_19,CA19CD,CA19NM,All_Ages_2019,DZ11NM_16,CA16CD,CA16NM,All_Ages_2016,_merge
DZ2011CD,,,,,,,,,
S01006506,Culter - 01,S12000033,Aberdeen City,867,Culter - 01,S12000033,Aberdeen City,897,both
S01006507,Culter - 02,S12000033,Aberdeen City,780,Culter - 02,S12000033,Aberdeen City,807,both
S01006508,Culter - 03,S12000033,Aberdeen City,615,Culter - 03,S12000033,Aberdeen City,640,both
S01006509,Culter - 04,S12000033,Aberdeen City,508,Culter - 04,S12000033,Aberdeen City,555,both
S01006510,Culter - 05,S12000033,Aberdeen City,649,Culter - 05,S12000033,Aberdeen City,671,both


C:\Users\shavar\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


LSOA11CD
E01011949   -0.002041
E01011950   -0.033285
E01011951   -0.002198
E01011952   -0.002861
E01011953   -0.001146
Name: pgr19_16, dtype: float64

DZ2011CD
S01006506   -0.011339
S01006507   -0.011343
S01006508   -0.013282
S01006509   -0.029496
S01006510   -0.011112
Name: pgr19_16, dtype: float64

In [98]:
# calculate Land Consumption Rate (LCR) = ((Vpresent - Vpast)/Vpast)/(t)
     # where : Vpresent is total built-up area in current year
            # Vpast is total built-up area in past year
            # (t) is number of years between Vpresnt and Vpast (or length in years of the period considered)

# read land_area data 
    # make sure all columns with numbers should have float dtype by using (df = pd.read_csv(<PATH>, thousands=',')  
land_area16gb = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\OS\lsoa2016_landcover_area_GB.csv', thousands=',')
land_area19_sc = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\OS\lsoa2019_landcover_area_sc.csv', thousands=',')
land_area19_ew = pd.read_csv(r'R:\SDGs\sdg_11_3_1\data\OS\lsoa2019_landcover_area_ew.csv', thousands=',')


    # clean columns names by removing whitespaces before and afer column names
land_area16gb.columns = land_area16gb.columns.str.strip()
land_area19_sc.columns = land_area19_sc.columns.str.strip()
land_area19_ew.columns = land_area19_ew.columns.str.strip()

    #inspect imported data visually
display(land_area16gb.head())
display(land_area19_sc.head())
display(land_area19_ew.head())

    #split land_area file for 2016 into England and Wales (EW) and Scotland (SC) (only necessary step because OS data is mixed between OAs for SC and LSOAs for EW)
filt_ew=(land_area16gb['CTRY']=='E') | (land_area16gb['CTRY']=='W') # filter to separate EW data
filt_sc=(land_area16gb['CTRY']=='S') # # filter to separate SC data

land_area16_ew=land_area16gb.loc[filt_ew]
land_area16_sc=land_area16gb.loc[filt_sc]

    #rename SC column names to SC format 
land_area16_sc.rename({'LSOA11CD':'DataZone2011Code','LSOA11NM':'DataZone2011Name','LAD16CD':'CouncilArea2016Code','LAD16NM':'CouncilArea2016Name'}, axis=1, inplace=True)
    #write 2016 land_area files to file
land_area16_ew.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area16_ew.csv')
land_area16_sc.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area16_sc.csv')

display(land_area16_ew.head())
display(land_area16_sc.head())

    #filter landcover type = manmade for 2019 data
filt_19ewmm = (land_area19_ew['landcover_type']=='Manmade') # filter 'manmade' landcover for EW
land_area19_ewmm = land_area19_ew.loc[filt_19ewmm]
land_area19_ewmm.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area19_ewmm.csv')

filt_19scmm = (land_area19_sc['landcover_type']=='Manmade') # filter 'manmade' landcover for SC
land_area19_scmm = land_area19_sc[['CTRY','DataZone2011Code','CouncilArea2011Code','landcover_type','area_2019']].loc[filt_19scmm]
   # aggregate Scotland OAs to DZs (equavalent of LSOA in EW)
land_area19_scdzmm = land_area19_scmm.groupby(['CouncilArea2011Code','DataZone2011Code','landcover_type']).sum()
land_area19_scdzmm.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area19_scdzmm.csv') #write to csv

display(land_area19_ewmm.head()) # EW LSOA 'manmade' land_area 
display(land_area19_scdzmm.head()) # EW DZ 'manmade' land_area

# calculate Land Consumption Rate (LCR) = ((Vpresent - Vpast)/Vpast)/(t)
      
    #read land_area files and merge to calculate LCR
land_area16_ew = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area16_ew.csv').set_index('LSOA11CD')
land_area16_sc = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area16_sc.csv').set_index('DataZone2011Code')
land_area19_scdzmm = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area19_scdzmm.csv').set_index('DataZone2011Code')
land_area19_ewmm = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\land_area19_ewmm.csv').set_index('LSOA11CD')
                             
    # merge 2019 and 2016 land_area files for EW and SC to create one table for each 
land_area19_16ewmm = pd.merge(land_area19_ewmm, land_area16_ew, on='LSOA11CD', how='outer', indicator=True, suffixes=('_19','_16'))
land_area19_16scmm = pd.merge(land_area19_scdzmm, land_area16_sc, on='DataZone2011Code', how='outer', indicator=True, suffixes=('_19','_16'))                        

 
    #check and remove unwanted columns
#land_area19_16ewmm.columns
#land_area19_16scmm.columns

#calculate Land Consumption Rate (LCR) = ((Vpresent - Vpast)/Vpast)/(t) 
land_area19_16ewmm['LCR2016_19'] = (land_area19_16ewmm['area_2019']-land_area19_16ewmm['area_2016'])/3
land_area19_16scmm['LCR2016_19'] = (land_area19_16scmm['area_2019']-land_area19_16scmm['area_2016'])/3      
                                                                   
display(land_area19_16ewmm.head())   
display(land_area19_16scmm.head())

    #write merge output to file
land_area19_16ewmm.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\lcr19_16ewmm.csv')
land_area19_16scmm.to_csv(r'R:\SDGs\sdg_11_3_1\intermediate\lcr19_16scmm.csv')



,CTRY,LSOA11CD,LSOA11NM,LAD16CD,LAD16NM,landcover_type,area_2013,area_2016,LCR2013_16
0,E,E01000001,City of London 001A,E09000001,City of London,Manmade,116142.47,116507.39,121.640333
1,E,E01000002,City of London 001B,E09000001,City of London,Manmade,206246.67,199430.64,-2272.011000
2,E,E01000003,City of London 001C,E09000001,City of London,Manmade,54174.38,51603.17,-857.070667
3,E,E01000005,City of London 001E,E09000001,City of London,Manmade,177250.86,169049.75,-2733.702667
4,E,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,Manmade,80508.63,80508.63,0.000000


,CTRY,OutputArea2011Code,DataZone2011Code,CouncilArea2011Code,landcover_type,area_2019
0,S,S00088956,S01006732,S12000033,Manmade,18196.63461
1,S,S00088956,S01006732,S12000033,Multiple,22927.20123
2,S,S00088956,S01006732,S12000033,Natural,41132.70593
3,S,S00088957,S01006732,S12000033,Manmade,428637.51190
4,S,S00088957,S01006732,S12000033,Multiple,46287.15487


,CTRY,LSOA11CD,LSOA11NM,LAD19CD,LAD19NM,landcover_type,area_2019
0,E,E01000001,City of London 001A,E09000001,City of London,Manmade,142088.128100
1,E,E01000001,City of London 001A,E09000001,City of London,Multiple,578.593375
2,E,E01000001,City of London 001A,E09000001,City of London,Natural,15199.874110
3,E,E01000001,City of London 001A,E09000001,City of London,Unclassified,328.860248
4,E,E01000002,City of London 001B,E09000001,City of London,Manmade,142518.964700


C:\Users\shavar\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,CTRY,LSOA11CD,LSOA11NM,LAD16CD,LAD16NM,landcover_type,area_2013,area_2016,LCR2013_16
0,E,E01000001,City of London 001A,E09000001,City of London,Manmade,116142.47,116507.39,121.640333
1,E,E01000002,City of London 001B,E09000001,City of London,Manmade,206246.67,199430.64,-2272.011000
2,E,E01000003,City of London 001C,E09000001,City of London,Manmade,54174.38,51603.17,-857.070667
3,E,E01000005,City of London 001E,E09000001,City of London,Manmade,177250.86,169049.75,-2733.702667
4,E,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,Manmade,80508.63,80508.63,0.000000


,CTRY,DataZone2011Code,DataZone2011Name,CouncilArea2016Code,CouncilArea2016Name,landcover_type,area_2013,area_2016,LCR2013_16
32844,S,S01006506,Culter - 01,S12000033,Aberdeen City,Manmade,147445.63,148827.80,460.722667
32845,S,S01006507,Culter - 02,S12000033,Aberdeen City,Manmade,69841.78,72180.40,779.539333
32846,S,S01006508,Culter - 03,S12000033,Aberdeen City,Manmade,59986.96,61711.02,574.688333
32847,S,S01006509,Culter - 04,S12000033,Aberdeen City,Manmade,37436.70,37909.27,157.524667
32848,S,S01006510,Culter - 05,S12000033,Aberdeen City,Manmade,58443.74,64195.59,1917.280333


,CTRY,LSOA11CD,LSOA11NM,LAD19CD,LAD19NM,landcover_type,area_2019
0,E,E01000001,City of London 001A,E09000001,City of London,Manmade,142088.12810
4,E,E01000002,City of London 001B,E09000001,City of London,Manmade,142518.96470
8,E,E01000003,City of London 001C,E09000001,City of London,Manmade,34955.80132
12,E,E01000005,City of London 001E,E09000001,City of London,Manmade,194423.47860
16,E,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,Manmade,71954.59214


area_2019
CouncilArea2011Code DataZone2011Code landcover_type               
S12000005           S01007409        Manmade         946673.043380
                    S01007410        Manmade          48290.588589
                    S01007411        Manmade          88043.720622
                    S01007412        Manmade          40132.176855
                    S01007413        Manmade          91526.202456

,Unnamed: 0_19,CTRY_19,LSOA11NM_19,LAD19CD,LAD19NM,landcover_type_19,area_2019,Unnamed: 0_16,CTRY_16,LSOA11NM_16,LAD16CD,LAD16NM,landcover_type_16,area_2013,area_2016,LCR2013_16,_merge,LCR2016_19
LSOA11CD,,,,,,,,,,,,,,,,,,
E01000001,0,E,City of London 001A,E09000001,City of London,Manmade,142088.12810,0,E,City of London 001A,E09000001,City of London,Manmade,116142.47,116507.39,121.640333,both,8526.912700
E01000002,4,E,City of London 001B,E09000001,City of London,Manmade,142518.96470,1,E,City of London 001B,E09000001,City of London,Manmade,206246.67,199430.64,-2272.011000,both,-18970.558433
E01000003,8,E,City of London 001C,E09000001,City of London,Manmade,34955.80132,2,E,City of London 001C,E09000001,City of London,Manmade,54174.38,51603.17,-857.070667,both,-5549.122893
E01000005,12,E,City of London 001E,E09000001,City of London,Manmade,194423.47860,3,E,City of London 001E,E09000001,City of London,Manmade,177250.86,169049.75,-2733.702667,both,8457.909533
E01000006,16,E,Barking and Dagenham 016A,E09000002,Barking and Dagenham,Manmade,71954.59214,4,E,Barking and Dagenham 016A,E09000002,Barking and Dagenham,Manmade,80508.63,80508.63,0.000000,both,-2851.345953


,CouncilArea2011Code,landcover_type_19,area_2019,Unnamed: 0,CTRY,DataZone2011Name,CouncilArea2016Code,CouncilArea2016Name,landcover_type_16,area_2013,area_2016,LCR2013_16,_merge,LCR2016_19
DataZone2011Code,,,,,,,,,,,,,,
S01007409,S12000005,Manmade,946673.043380,33747,S,Tullibody South - 01,S12000005,Clackmannanshire,Manmade,888517.45,913566.35,8349.632000,both,11035.564460
S01007410,S12000005,Manmade,48290.588589,33748,S,Tullibody South - 02,S12000005,Clackmannanshire,Manmade,47183.50,47203.27,6.587667,both,362.439530
S01007411,S12000005,Manmade,88043.720622,33749,S,Tullibody South - 03,S12000005,Clackmannanshire,Manmade,92131.24,89510.69,-873.514333,both,-488.989793
S01007412,S12000005,Manmade,40132.176855,33750,S,Tullibody South - 04,S12000005,Clackmannanshire,Manmade,37056.36,37056.36,0.000000,both,1025.272285
S01007413,S12000005,Manmade,91526.202456,33751,S,Tullibody South - 05,S12000005,Clackmannanshire,Manmade,84660.05,84990.21,110.051333,both,2178.664152


In [99]:
# calculate Ratio of land consumption rate to population growth rate (LCRPGR)

            # LCRPGR = (Land Consumption Rate)/(Population Growth Rate)
    
# read population (PGR) data files and land_area (LCR) data files
pgr_19_16ew = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\pgr_19_16ew.csv')
pgr_19_16sc = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\pgr_19_16sc.csv')
lcr19_16ewmm = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\lcr19_16ewmm.csv')
lcr19_16scmm = pd.read_csv(r'R:\SDGs\sdg_11_3_1\intermediate\lcr19_16scmm.csv')

display(pgr_19_16ew.columns)
display(pgr_19_16sc.columns)
display(lcr19_16ewmm.columns)
display(lcr19_16scmm.columns)

Index(['LSOA11CD', 'LSOA11NM_19', 'LAD19CD', 'LAD19NM', 'All_Ages_2019',
       'LSOA11NM_16', 'LAD16CD', 'LAD16NM', 'All_Ages_2016', '_merge',
       'pgr19_16'],
      dtype='object')

Index(['DZ2011CD', 'DZ11NM_19', 'CA19CD', 'CA19NM', 'All_Ages_2019',
       'DZ11NM_16', 'CA16CD', 'CA16NM', 'All_Ages_2016', '_merge', 'pgr19_16'],
      dtype='object')

Index(['LSOA11CD', 'Unnamed: 0_19', 'CTRY_19', 'LSOA11NM_19', 'LAD19CD',
       'LAD19NM', 'landcover_type_19', 'area_2019', 'Unnamed: 0_16', 'CTRY_16',
       'LSOA11NM_16', 'LAD16CD', 'LAD16NM', 'landcover_type_16', 'area_2013',
       'area_2016', 'LCR2013_16', '_merge', 'LCR2016_19'],
      dtype='object')

Index(['DataZone2011Code', 'CouncilArea2011Code', 'landcover_type_19',
       'area_2019', 'Unnamed: 0', 'CTRY', 'DataZone2011Name',
       'CouncilArea2016Code', 'CouncilArea2016Name', 'landcover_type_16',
       'area_2013', 'area_2016', 'LCR2013_16', '_merge', 'LCR2016_19'],
      dtype='object')

In [102]:
# remove unwanted columns and merge PGR and LCR tables to calculate LCRPGR
pgr_19_16ew2 = pgr_19_16ew[['LSOA11CD', 'LAD19CD', 'LAD19NM',  'LAD16CD', 'LAD16NM', 'All_Ages_2019','All_Ages_2016', 'pgr19_16']]
pgr_19_16sc2 = pgr_19_16sc[['DZ2011CD', 'DZ11NM_19', 'CA19CD', 'CA19NM',  'CA16CD', 'CA16NM', 'All_Ages_2019','All_Ages_2016', 'pgr19_16']]
lcr19_16ewmm2 = lcr19_16ewmm[['LSOA11CD', 'LSOA11NM_19', 'LAD19CD',
       'LAD19NM', 'landcover_type_19',  'LAD16CD', 'LAD16NM', 'area_2013',
       'area_2016', 'area_2019', 'LCR2013_16', 'LCR2016_19']]
lcr19_16scmm2 = lcr19_16scmm[['DataZone2011Code', 'DataZone2011Name', 'CouncilArea2011Code', 'CouncilArea2016Code', 'CouncilArea2016Name', 'landcover_type_19',
       'area_2013', 'area_2016', 'area_2019', 'LCR2013_16', 'LCR2016_19']]
#display(pgr_19_16ew2.head())
#display(pgr_19_16sc2.head())
#display(lcr19_16ewmm2.head())
#display(lcr19_16ewmm2.head())

    #merge PGR and LCR tables
lcrpgr_19_16ew = pd.merge(pgr_19_16ew2, lcr19_16ewmm2, on='LSOA11CD', how='outer', indicator=True, suffixes=('_pgr', '_lcr'))
lcrpgr_19_16sc = pd.merge(pgr_19_16sc2, lcr19_16scmm2, left_on='DZ2011CD',right_on='DataZone2011Code', how='outer', indicator=True, suffixes=('_pgr', '_lcr'))                        

#display(lcrpgr_19_16ew.head())
#display(lcrpgr_19_16sc.head())

# calculate LCRPGR
lcrpgr_19_16ew['LCRPGR_19_16'] = lcrpgr_19_16ew['LCR2016_19']/lcrpgr_19_16ew['pgr19_16']
lcrpgr_19_16sc['LCRPGR_19_16'] = lcrpgr_19_16sc['LCR2016_19']/lcrpgr_19_16sc['pgr19_16']

display(lcrpgr_19_16ew.head())
display(lcrpgr_19_16sc.head())

#write output to file
lcrpgr_19_16ew.to_csv(r'R:\SDGs\sdg_11_3_1\analysis_outputs\lcrpgr_19_16ew.csv')
lcrpgr_19_16sc.to_csv(r'R:\SDGs\sdg_11_3_1\analysis_outputs\lcrpgr_19_16sc.csv')

,LSOA11CD,LAD19CD_pgr,LAD19NM_pgr,LAD16CD_pgr,LAD16NM_pgr,All_Ages_2019,All_Ages_2016,pgr19_16,LSOA11NM_19,LAD19CD_lcr,...,landcover_type_19,LAD16CD_lcr,LAD16NM_lcr,area_2013,area_2016,area_2019,LCR2013_16,LCR2016_19,_merge,LCRPGR_19_16
0,E01011949,E06000001,Hartlepool,E06000001,Hartlepool,1954,1966,-0.002041,Hartlepool 009A,E06000001,...,Manmade,E06000001,Hartlepool,145589.05,143845.34,143996.52430,-581.235333,50.394767,both,-2.469336e+04
1,E01011950,E06000001,Hartlepool,E06000001,Hartlepool,1257,1389,-0.033285,Hartlepool 008A,E06000001,...,Manmade,E06000001,Hartlepool,92947.61,93001.94,80679.05422,18.109667,-4107.628593,both,1.234064e+05
2,E01011951,E06000001,Hartlepool,E06000001,Hartlepool,1209,1217,-0.002198,Hartlepool 007A,E06000001,...,Manmade,E06000001,Hartlepool,110023.49,109992.19,109680.58270,-10.432667,-103.869100,both,4.724729e+04
3,E01011952,E06000001,Hartlepool,E06000001,Hartlepool,1740,1755,-0.002861,Hartlepool 002A,E06000001,...,Manmade,E06000001,Hartlepool,224785.57,225045.48,217940.92760,86.635000,-2368.184133,both,8.276753e+05
4,E01011953,E06000001,Hartlepool,E06000001,Hartlepool,2033,2040,-0.001146,Hartlepool 002B,E06000001,...,Manmade,E06000001,Hartlepool,170752.73,178479.10,238494.97400,2575.456000,20005.291333,both,-1.746032e+07


,DZ2011CD,DZ11NM_19,CA19CD,CA19NM,CA16CD,CA16NM,All_Ages_2019,All_Ages_2016,pgr19_16,DataZone2011Code,...,CouncilArea2016Code,CouncilArea2016Name,landcover_type_19,area_2013,area_2016,area_2019,LCR2013_16,LCR2016_19,_merge,LCRPGR_19_16
0,S01006506,Culter - 01,S12000033,Aberdeen City,S12000033,Aberdeen City,867,897,-0.011339,S01006506,...,S12000033,Aberdeen City,Manmade,147445.63,148827.80,195653.827297,460.722667,15608.675766,both,-1.376552e+06
1,S01006507,Culter - 02,S12000033,Aberdeen City,S12000033,Aberdeen City,780,807,-0.011343,S01006507,...,S12000033,Aberdeen City,Manmade,69841.78,72180.40,78907.849546,779.539333,2242.483182,both,-1.976932e+05
2,S01006508,Culter - 03,S12000033,Aberdeen City,S12000033,Aberdeen City,615,640,-0.013282,S01006508,...,S12000033,Aberdeen City,Manmade,59986.96,61711.02,69683.064996,574.688333,2657.348332,both,-2.000719e+05
3,S01006509,Culter - 04,S12000033,Aberdeen City,S12000033,Aberdeen City,508,555,-0.029496,S01006509,...,S12000033,Aberdeen City,Manmade,37436.70,37909.27,37909.306610,157.524667,0.012203,both,-4.137347e-01
4,S01006510,Culter - 05,S12000033,Aberdeen City,S12000033,Aberdeen City,649,671,-0.011112,S01006510,...,S12000033,Aberdeen City,Manmade,58443.74,64195.59,66217.962882,1917.280333,674.124294,both,-6.066557e+04
